<a href="https://www.kaggle.com/code/yaaangzhou/tutorial-for-beginners-train-a-stackingmodel?scriptVersionId=133536752" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e16/sample_submission.csv
/kaggle/input/playground-series-s3e16/train.csv
/kaggle/input/playground-series-s3e16/test.csv
/kaggle/input/crab-age-prediction/CrabAgePrediction.csv


The idea comes from this notebook: https://www.kaggle.com/code/oscarm524/ps-s3-ep16-eda-modeling-submission

# Imports

In [2]:
pip install scikit-lego

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.9/230.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns
import plotly.express as px

from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV, RepeatedKFold, RepeatedStratifiedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.inspection import PartialDependenceDisplay
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklego.linear_model import LADRegression

## **We also use the original data to train the model.**

In [4]:
train = pd.read_csv('../input/playground-series-s3e16/train.csv')
test = pd.read_csv('../input/playground-series-s3e16/test.csv')
original = pd.read_csv('../input/crab-age-prediction/CrabAgePrediction.csv')
submission = pd.read_csv('../input/playground-series-s3e16/sample_submission.csv')

# Feature Engineering

**The EDA part is omitted here, we will use all variables and create some new ones. Also, we need to deal with categorical variables.**

In [5]:
le = LabelEncoder()

# the data is original or not
train['generated'] = 1
original['generated'] = 0
test['generated'] = 1

train.drop(columns = 'id', axis = 1, inplace = True)

train = pd.concat([train, original], axis = 0).reset_index(drop = True)
train['Sex'] = le.fit_transform(train['Sex'])

In [6]:
X = train.drop(columns = ['Age'], axis = 1)
Y = train['Age']

X['Meat Yield'] = X['Shucked Weight'] / (X['Weight'] + X['Shell Weight'])
X['Shell Ratio'] = X['Shell Weight'] / X['Weight']
X['Weight_to_Shucked_Weight'] = X['Weight'] / X['Shucked Weight']
X['Viscera Ratio'] = X['Viscera Weight'] / X['Weight']

In [7]:
X.head()

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,generated,Meat Yield,Shell Ratio,Weight_to_Shucked_Weight,Viscera Ratio
0,1,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,1,0.341056,0.288160,2.276169,0.229452
1,1,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,1,0.327179,0.326531,2.304075,0.223129
2,2,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,1,0.360685,0.268879,2.185000,0.224256
3,0,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,1,0.310017,0.296027,2.488858,0.216970
4,1,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,1,0.409598,0.255630,1.944379,0.193548


# CV + Modeling

In [8]:
# Test data
test_baseline = test.drop(columns = ['id'], axis = 1)

test_baseline['Sex'] = le.transform(test_baseline['Sex'])
test_baseline['Meat Yield'] = test_baseline['Shucked Weight'] / (test_baseline['Weight'] + test_baseline['Shell Weight'])
test_baseline['Shell Ratio'] = test_baseline['Shell Weight'] / test_baseline['Weight']
test_baseline['Weight_to_Shucked_Weight'] = test_baseline['Weight'] / test_baseline['Shucked Weight']
test_baseline['Viscera Ratio'] = test_baseline['Viscera Weight'] / test_baseline['Weight']

In [9]:
skf = KFold(n_splits = 5, random_state = 42, shuffle = True)
gb_cv_scores, gb_preds = [], []
hist_cv_scores, hist_preds = [], []
lgb_cv_scores, lgb_preds = [], []
xgb_cv_scores, xgb_preds = [], []
cat_cv_scores, cat_preds = [], []
ens_cv_scores, ens_preds = [], []

In [10]:
models = {
        'GradientBoosting': {
        'model': GradientBoostingRegressor,
        'name': 'GradientBoosting',
        'params': {
            'loss': 'absolute_error',
            'n_estimators': 1000,
            'max_depth': 8,
            'learning_rate': 0.01,
            'min_samples_split': 10,
            'min_samples_leaf': 20,
            'random_state': 42
        },
        'features': ['Sex', 'Length', 'Diameter', 'Height', 'Weight', 'Shucked Weight', 'Viscera Weight', 'Shell Weight', 'generated']
    },
    'HistGradientBoosting': {
        'model': HistGradientBoostingRegressor,
        'name': 'HistGradientBoosting',
        'params': {
            'loss': 'absolute_error',
            'l2_regularization': 0.01,
            'early_stopping': False,
            'learning_rate': 0.01,
            'max_iter': 1000,
            'max_depth': 15,
            'max_bins': 255,
            'min_samples_leaf': 70,
            'max_leaf_nodes': 115,
            'random_state': 42
        },
        'features': ['Sex', 'Length', 'Diameter', 'Height', 'Weight', 'Shucked Weight','Viscera Weight', 'Shell Weight', 'generated', 'Meat Yield','Shell Ratio', 'Weight_to_Shucked_Weight', 'Viscera Ratio']
    },
    
    'LightGBM': {
        'model': LGBMRegressor,
        'name': 'LightGBM',
        'params': {
           'objective' : 'mae', 
            'n_estimators' : 1000,
            'max_depth' : 15,
            'learning_rate' : 0.01,
            'num_leaves' : 105, 
            'reg_alpha' : 8, 
            'reg_lambda' : 3, 
            'subsample' : 0.6, 
            'colsample_bytree' : 0.8,
            'random_state' : 42     
        },
        'features': ['Sex', 'Length', 'Diameter', 'Height', 'Weight', 'Shucked Weight','Viscera Weight', 'Shell Weight', 'generated', 'Meat Yield','Shell Ratio', 'Weight_to_Shucked_Weight', 'Viscera Ratio']
    },
    
    'XGBoost': {
        'model':XGBRegressor,
        'name': 'XGBoost',
        'params': {
         'objective' : 'reg:pseudohubererror',
         'tree_method' : 'hist',
         'colsample_bytree' : 0.9, 
         'gamma' : 0.65, 
         'learning_rate' : 0.01, 
         'max_depth' : 7, 
         'min_child_weight' : 20, 
         'n_estimators' : 1500,
         'subsample' : 0.7,
         'random_state' : 42               
            },
        'features': ['Sex', 'Length', 'Diameter', 'Height', 'Weight', 'Shucked Weight','Viscera Weight', 'Shell Weight', 'generated', 'Meat Yield','Shell Ratio', 'Weight_to_Shucked_Weight', 'Viscera Ratio']

    },
    
    'CatBoost': {
        'model': CatBoostRegressor,
        'name': 'CatBoost',
        'params': {
            'loss_function' : 'MAE',
            'iterations' : 1000,
            'learning_rate' : 0.08,
            'depth' : 10, 
            'random_strength' : 0.2,
            'bagging_temperature' : 0.7,
            'border_count' : 254,
            'l2_leaf_reg' : 0.001,
            'verbose' : False,
            'grow_policy' : 'Lossguide',
            'task_type' : 'CPU',
            'random_state' : 42         
        },
        'features': ['Sex','Length','Diameter','Height','Weight','Shucked Weight','Viscera Weight','Shell Weight','generated','Meat Yield','Shell Ratio','Weight_to_Shucked_Weight'] 
    },

    'Ensemble': {
        'model': LADRegression,
        'name': 'Ensemble',
        'params': {
            'fit_intercept' : False, 
            'positive' : True
        },
    
    }
}

In [11]:
skf = KFold(n_splits = 3, random_state = 42, shuffle = True)

def run_model(x,y,models):
    

    for i,(train_idx, test_idx) in enumerate(skf.split(x, y)):

        ens_pred = {}
        
        for model in models:
            model_outputs = {}
            model_pred = {}
            
            if models[model]['name'] != 'Ensemble':
                cv_score = []
                X_train, X_test = X.iloc[train_idx][models[model]['features']], X.iloc[test_idx][models[model]['features']]
                Y_train, Y_test = Y.iloc[train_idx], Y.iloc[test_idx]
                test_data = test_baseline[models[model]['features']]
                
                print('--'*25)

                print('Model:', models[model]['name'])
                
                current_model = models[model]['model'](**models[model]['params'])

                current_model.fit(X_train, Y_train)

                pred = current_model.predict(X_test[X_test['generated'] == 1])
                model_outputs[models[model]['name'] ] = pred

                score_fold = mean_absolute_error(Y_test[X_test['generated'] == 1], pred)
                cv_score.append(score_fold)

                test_pred = current_model.predict(test_data)
                model_pred[models[model]['name']] = test_pred
                
                print('Fold', i, '==>', models[model]['name'], 'oof MAE is ==>', score_fold)

            if models[model]['name'] == 'Ensemble':
                model_outputs_df = pd.DataFrame(model_outputs)
                Ens_x =  model_outputs_df
                Ens_y = Y_test[X_test['generated'] == 1]
                model_outputs_df_test = pd.DataFrame(model_pred)
                
                print('--'*25)
                print('Model:', models[model]['name'])
                current_model = models[model]['model'](**models[model]['params'])

                current_model.fit(Ens_x, Ens_y)

                pred = current_model.predict(Ens_x)

                pred_test = current_model.predict(model_outputs_df_test)

                ens_pred[models[model]['name']+str(i)] = pred_test

                score_fold = mean_absolute_error(Ens_y, pred)

                print('Fold', i, '==>', models[model]['name'], 'oof MAE is ==>', score_fold)



In [12]:
skf = KFold(n_splits=2, random_state=42, shuffle=True)

ens_pred = {}
def run_model(x, y, models):

    model_outputs = {}  # 在循环外部定义model_outputs字典
    model_pred = {}  # 在循环外部定义model_pred字典
    
    for i, (train_idx, test_idx) in enumerate(skf.split(x, y)):
        for model in models:
            if models[model]['name'] != 'Ensemble':
                cv_score = []
                X_train, X_test = x.iloc[train_idx][models[model]['features']], x.iloc[test_idx][models[model]['features']]
                Y_train, Y_test = y.iloc[train_idx], y.iloc[test_idx]
                test_data = test_baseline[models[model]['features']]
                
                print('--' * 25)
                print('Model:', models[model]['name'])
                
                current_model = models[model]['model'](**models[model]['params'])
                current_model.fit(X_train, Y_train)

                pred = current_model.predict(X_test[X_test['generated'] == 1])
                model_outputs[models[model]['name']] = pred

                score_fold = mean_absolute_error(Y_test[X_test['generated'] == 1], pred)
                cv_score.append(score_fold)

                test_pred = current_model.predict(test_data)
                model_pred[models[model]['name']] = test_pred
                
                print('Fold', i, '==>', models[model]['name'], 'oof MAE is ==>', score_fold)

            if models[model]['name'] == 'Ensemble':
                model_outputs_df = pd.DataFrame(model_outputs)
                Ens_x = model_outputs_df
                Ens_y = Y_test[X_test['generated'] == 1]
                model_outputs_df_test = pd.DataFrame(model_pred)
                
                print('--' * 25)
                print('Model:', models[model]['name'])
                current_model = models[model]['model'](**models[model]['params'])
                current_model.fit(Ens_x, Ens_y)

                pred = current_model.predict(Ens_x)
                pred_test = current_model.predict(model_outputs_df_test)

                ens_pred[models[model]['name'] + str(i)] = pred_test

                score_fold = mean_absolute_error(Ens_y, pred)
                print('Fold', i, '==>', models[model]['name'], 'oof MAE is ==>', score_fold)
    
    return ens_pred


In [13]:
run_model(X,Y,models)

--------------------------------------------------
Model: GradientBoosting
Fold 0 ==> GradientBoosting oof MAE is ==> 1.3602650397459801
--------------------------------------------------
Model: HistGradientBoosting
Fold 0 ==> HistGradientBoosting oof MAE is ==> 1.3599870332715633
--------------------------------------------------
Model: LightGBM
Fold 0 ==> LightGBM oof MAE is ==> 1.358061066460425
--------------------------------------------------
Model: XGBoost
Fold 0 ==> XGBoost oof MAE is ==> 1.369699996464388
--------------------------------------------------
Model: CatBoost
Fold 0 ==> CatBoost oof MAE is ==> 1.364902991367803
--------------------------------------------------
Model: Ensemble
Fold 0 ==> Ensemble oof MAE is ==> 1.3532556966965656
--------------------------------------------------
Model: GradientBoosting
Fold 1 ==> GradientBoosting oof MAE is ==> 1.3643674374707864
--------------------------------------------------
Model: HistGradientBoosting
Fold 1 ==> HistGradient

{'Ensemble0': array([ 7.39523351,  7.87505827, 10.63407896, ..., 12.78047625,
         9.70842644, 11.12365898]),
 'Ensemble1': array([ 7.31930179,  7.98104317,  9.67041842, ..., 13.4575009 ,
         9.92334223, 11.32594828])}

# Final submit

In [14]:
ens_preds_test = pd.DataFrame(ens_pred).apply(np.mean, axis = 1)

submission['Age'] = round(ens_preds_test).astype(int)
submission.to_csv('submission.csv', index = False)

In [15]:
submission

,id,Age
0,74051,7
1,74052,8
2,74053,10
3,74054,9
4,74055,7
...,...,...
49363,123414,9
49364,123415,8
49365,123416,13
49366,123417,10
